In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from dechorate import constants

# Load RIRs

In [9]:
path_to_hdf = Path('..', 'dEchorate_rirs.hdf5')
dset = h5py.File(path_to_hdf, mode='r')

signals = list(dset.keys())
signal = signals[0]
print('Signals', signals)

rooms = list(dset[signals[0]].keys())
print('Rooms', rooms)

sources = list(dset[signals[0]][rooms[0]].keys())
print(sources)

Signals ['rir']
Rooms ['000000', '000001', '000010', '000100', '001000', '010000', '011000', '011100', '011110', '011111', '020002']
['1', '2', '3', '4', '5', '6', '7', '8', '9']


In [10]:
path_to_delay_note = Path('..','data', 'dEchorate_database.csv')
df = pd.read_csv(path_to_delay_note)
df

,Unnamed: 0,filename,src_id,src_ch,src_type,src_signal,src_pos_x,src_pos_y,src_pos_z,room_code,...,mic_type,mic_id,mic_ch,mic_pos_x,mic_pos_y,mic_pos_z,array_id,array_bar_x,array_bar_y,array_bar_z
0,0,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,1.0,33.0,0.857713,3.909900,1.039,1.0,0.972,3.954,1.039
1,1,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,2.0,34.0,0.895031,3.924300,1.039,1.0,0.972,3.954,1.039
2,2,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,3.0,35.0,0.941679,3.942300,1.039,1.0,0.972,3.954,1.039
3,3,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,4.0,36.0,1.002321,3.965700,1.039,1.0,0.972,3.954,1.039
4,4,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,5.0,37.0,1.095616,4.001700,1.039,1.0,0.972,3.954,1.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10907,10907,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,27.0,59.0,3.097404,3.396043,1.460,6.0,3.031,3.445,1.460
10908,10908,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,28.0,60.0,3.057159,3.425714,1.460,6.0,3.031,3.445,1.460
10909,10909,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,29.0,61.0,3.004841,3.464286,1.460,6.0,3.031,3.445,1.460
10910,10910,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,30.0,62.0,2.924352,3.523628,1.460,6.0,3.031,3.445,1.460


In [20]:
D = len(rooms)
J = len(sources)
I = 30
Fs = constants['Fs']

rirs = np.zeros([2*Fs, I, J, D])

for r, room in enumerate(rooms):

    for j, src in enumerate(srcs):

        group = f'/{signal}/{room}/{src}'
        data = np.asarray(dset[group])
        
        rirs[:,:,j,r] = data[:2*Fs,:-1]
        
print(rirs.shape)

(96000, 30, 9, 11)


# Visualize RIRs with Skyline